In [52]:
import glob 
import os
import cv2
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from albumentations.pytorch import ToTensorV2
from albumentations import Compose,Resize

from torch.utils.data import Dataset,DataLoader

In [14]:
train_files = glob.glob("train/*.jpg")
test_files = glob.glob(("test1/*jpg"))

In [15]:
len(train_files)
len(test_files)

12500

In [16]:


clean__train_files = []
clean__test_files = []
def clean_check(mode='train'):

    """
    Function to check any corrupted images are in the provided dataset

    Args:
    mode = train or test
    if train checks the train files and returns only the clean file and the same for the
    test or val mode.


    """
    files = train_files if mode=="train" else test_files
    save_list = clean__train_files if mode=="train" else clean__test_files
    for i in tqdm(range(len((files)))):
        img = cv2.imread(files[i])
        if img is None:
            raise(RuntimeWarning("NoneType Image Found and Excluded..."))
        else:
            save_list.append(train_files[i])
        

In [17]:
clean_check(mode='train')

  0%|          | 0/25000 [00:00<?, ?it/s]

In [18]:
clean_check(mode='test')

  0%|          | 0/12500 [00:00<?, ?it/s]

In [20]:
print("------ Count Check ------")
print(f"Number of Train Files:{len(clean__train_files)}/{len(train_files)}")
print(f"Number of Test Files:{len(clean__test_files)}/{len(test_files)}")

------ Count Check ------
Number of Train Files:25000/25000
Number of Test Files:12500/12500


In [29]:
clean__train_files[0].split("/")[1].split(".")[0]

'dog'

In [32]:
data = pd.DataFrame(index=np.arange(len(clean__train_files)))
data['Paths'] = clean__train_files
data['Labels'] = [1 if x.split("/")[1].split(".")[0]=="dog" else 0 for x in clean__train_files]

In [33]:
data.Labels.value_counts()

0    12500
1    12500
Name: Labels, dtype: int64

In [41]:
data['folds'] = -1
for i,(train_ind,val_ind) in enumerate(StratifiedKFold(n_splits=5).split(data.Paths.values,data.Labels.values)):
    data.loc[val_ind,'folds'] = i

In [42]:
data.folds.value_counts()

0    5000
1    5000
2    5000
3    5000
4    5000
Name: folds, dtype: int64

In [43]:
data.to_csv("data.csv",index=False)

In [49]:
d = data[data.folds != 0].reset_index(drop=True)

In [50]:
d

,Paths,Labels,folds
0,train/cat.8753.jpg,0,1
1,train/cat.169.jpg,0,1
2,train/cat.7556.jpg,0,1
3,train/cat.810.jpg,0,1
4,train/cat.1522.jpg,0,1
...,...,...,...
19995,train/dog.11497.jpg,1,4
19996,train/cat.9340.jpg,0,4
19997,train/cat.2860.jpg,0,4
19998,train/dog.391.jpg,1,4


## DataSets and DataLoaders

In [61]:
class dog_cat(Dataset):
    def __init__(self,df,mode="train",folds=0,transforms=None):
        super(dog_cat,self).__init__()
        self.df = df
        self.mode = mode
        self.folds = folds
        self.transforms = transforms

        if self.mode == "train":
            self.data = self.df[self.df.folds != self.folds].reset_index(drop=True)
            
        else:
            self.data = self.df[self.df.folds == self.folds].reset_index(drop=True)
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self,idx):

        img = cv2.imread(self.data.loc[idx,"Paths"])
        label = self.data.loc[idx,'Labels']

        if self.transforms is not None:
            image = self.transforms(image=img)['image']

        
        return image,label

In [62]:
train = dog_cat(data,transforms=Compose([Resize(512,512),ToTensorV2()]))

In [69]:
train_load = DataLoader(train,batch_size=len(train),num_workers=1,shuffle=True)
d = next(iter(train_load))
m,s = d[0].mean(),d[0].std()

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc8da7618b0>
Traceback (most recent call last):
  File "/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc8da7618b0>
Traceback (most recent call last):
  File "/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/

RuntimeError: DataLoader worker (pid(s) 25160) exited unexpectedly

In [4]:
import torch

In [19]:
y = torch.Tensor([0.5,0.8,0.3,0.4,0.7])
yhat = (y>0.5).float()
y = torch.Tensor([0.,1.,0.,0.,0.])
crt = (yhat == y).float().sum()

In [22]:
yhat.flatten()

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


tensor(0.)